In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [2]:
# Definições de parâmetros
num_variaveis = 2
w = [1, 2, 3, 4]
confianca = 0.75
X = [f'X {i+1}' for i in range(num_variaveis)]
Z = [f'Z {i+1}' for i in range(len(w))]
pi = [1 / len(w)] * len(w)
pi = pd.DataFrame(pi, index = w)

# Definindo os parâmetros cmáximos e mínimos
x_min = [0, 0]  # x_min para x_1 e x_2
x_max = [10, 10]  # x_max para x_1 e x_2
z_min = [0, 0, 0, 0]  # z_min para Z_1, Z_2, Z_3, Z_4
z_max = [1, 1, 1, 1]  # z_max para Z_1, Z_2, Z_3, Z_4

# Criando o modelo concreto
model = ConcreteModel('McCormick')

# Sets e Parâmetros
model.w = RangeSet(len(w))
model.i = RangeSet(len(x_min))  # Definindo para as variáveis x
model.confianca = Param(initialize=confianca)

# Inicializando parâmetros de pi (probabilidades) como dicionário
pi_values = {i+1: 1 / len(w) for i in range(len(w))}
model.pi = Param(model.w, initialize=pi_values)

# Inicializando parâmetros de x_min, x_max, z_min e z_max
model.x_min = Param(model.i, initialize={i+1: x_min[i] for i in range(len(x_min))})
model.x_max = Param(model.i, initialize={i+1: x_max[i] for i in range(len(x_max))})
model.z_min = Param(model.w, initialize={i+1: z_min[i] for i in range(len(z_min))})
model.z_max = Param(model.w, initialize={i+1: z_max[i] for i in range(len(z_max))})

# Definindo as variáveis de decisão
model.X = Var(model.i, domain=NonNegativeReals)
model.Z = Var(model.w, domain=Binary)
model.k = Var(model.w, model.i, domain=NonNegativeReals)

# Função objetivo (minimizar a soma das variáveis X)
model.obj = Objective(expr=sum(model.X[i] for i in model.i), sense=minimize)

# Restrições de McCormick (aquelas que ligam X e Z com a variável k)
model.Cor = ConstraintList()
for i in model.i:
    for w in model.w:
        model.Cor.add(expr=model.x_min[i]*model.Z[w] + model.X[i]*model.z_min[w] - model.x_min[i]*model.z_min[w] <= model.k[w,i])
        model.Cor.add(expr=model.x_max[i]*model.Z[w] + model.X[i]*model.z_max[w] - model.x_max[i]*model.z_max[w] <= model.k[w,i])
        model.Cor.add(expr=model.x_max[i]*model.Z[w] + model.X[i]*model.z_min[w] - model.x_max[i]*model.z_min[w] >= model.k[w,i])
        model.Cor.add(expr=model.x_min[i]*model.Z[w] + model.X[i]*model.z_max[w] - model.x_min[i]*model.z_max[w] >= model.k[w,i])

# Restrições de probabilidade (relacionadas à confiança)
model.pi_constraints = ConstraintList()
for w in model.w:
    model.pi_constraints.add(expr=sum(model.Z[w]*model.pi[w] for w in model.w) >= model.confianca)

# Restrições adicionais de k e Z
model.kZ_constraints = ConstraintList()
for w in model.w:
    model.kZ_constraints.add(expr=w*model.k[w,1] + model.k[w,2] - 7*model.Z[w] >= 0)

# Resolver o modelo
opt = SolverFactory('glpk')
opt.solve(model, tee=False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 3.5, 'Upper bound': 3.5, 'Number of objectives': 1, 'Number of constraints': 40, 'Number of variables': 14, 'Number of nonzeros': 92, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '3', 'Number of created subproblems': '3'}}, 'Error rc': 0, 'Time': 0.037819623947143555}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [3]:
# Exibindo resultados
for i in model.i:
    print(f"X {i+1}: {model.X[i].value}")
for w in model.w:
    print(f"Z {w}: {model.Z[w].value}")

X 2: 3.5
X 3: 2.42861286636753e-16
Z 1: 0.0
Z 2: 1.0
Z 3: 1.0
Z 4: 1.0
